In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import font_manager,rc 
import folium
import json
#한글 폰트 등록
font_location = "c:/Windows/fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)
#- 값 깨짐 방지
matplotlib.rcParams['axes.unicode_minus']=False

%matplotlib inline

In [2]:
비교=pd.read_excel('data/기존충전소공영(위경도포함).xlsx')
len(비교)

175

In [3]:
비교=비교.drop([비교.columns[1],비교.columns[4],비교.columns[5],비교.columns[9],비교.columns[10],비교.columns[11],비교.columns[12]],axis=1)
비교=비교.rename(columns={'X':'lon','Y':'lat'})
비교.head()

,address,lon,lat,주소구분,표준신주소,표준구주소,법정동명
0,서울특별시 강남구 개포동 13-2,127.079297,37.494945,구주소,서울특별시 강남구 개포로 623-1,서울특별시 강남구 개포동 13-2,개포동
1,서울특별시 강남구 개포로 25길 32,127.048515,37.481519,새주소,서울특별시 강남구 개포로25길 32,서울특별시 강남구 개포동 1266,개포동
2,"서울특별시 강남구 개포로 509, 지하주차장 1층 8구역(개포동 183-2)",127.068050,37.490000,새주소,서울특별시 강남구 개포로 509,서울특별시 강남구 개포동 183-1,개포동
3,"서울특별시 강남구 남부순환로 3104, 지하주차장 3층 M418구역(대치동 514)",127.072440,37.496162,새주소,서울특별시 강남구 남부순환로 3104,서울특별시 강남구 대치동 514,대치동
4,서울특별시 강남구 남부순환로 지하 3104학여울역 공영주차장 지하3층 6기둥(514...,127.071721,37.496715,새주소,서울특별시 강남구 남부순환로 지하 3104,서울특별시 강남구 대치동 514-3,대치동


In [4]:
주차장=pd.read_csv('data/중복제외공영주차장(위경도포함).csv')
주차장=주차장.drop([주차장.columns[1],주차장.columns[4],주차장.columns[5],주차장.columns[9],주차장.columns[10],주차장.columns[11],주차장.columns[12]],axis=1)
주차장=주차장.rename(columns={'X':'lon','Y':'lat'})
주차장.head()

,address,lon,lat,주소구분,표준신주소,표준구주소,법정동명
0,강남구 개포동 126-2,127.066489,37.477319,구주소,서울특별시 강남구 개포로 607,서울특별시 강남구 개포동 126-2,개포동
1,강남구 개포동 1266-0,127.048030,37.481443,구주소,서울특별시 강남구 개포로25길 32,서울특별시 강남구 개포동 1266,개포동
2,강남구 개포동 1273-0,127.052321,37.475602,구주소,서울특별시 강남구 개포로22길 87,서울특별시 강남구 개포동 1273,개포동
3,강남구 개포동 13-2,127.079297,37.494945,구주소,서울특별시 강남구 개포로 623-1,서울특별시 강남구 개포동 13-2,개포동
4,강남구 개포동 183-2,127.066202,37.489490,구주소,NaN,서울특별시 강남구 개포동 183-2,개포동


In [5]:
주차장2=pd.read_csv('data/중복제외공영주차장.csv')
주차장2=주차장2.drop(주차장2.columns[9:14],axis=1)
주차장2=주차장2.drop(주차장2.columns[3:5],axis=1)
주차장2.head()

,name,address,type,gu,dong,group,park_space
0,구룡산제2호(구),강남구 개포동 126-2,노외 주차장,강남구,개포동,동남권,132
1,논현로22길(시),강남구 개포동 1266-0,노외 주차장,강남구,개포동,동남권,97
2,포이초교 공영주차장(구),강남구 개포동 1273-0,노외 주차장,강남구,개포동,동남권,194
3,대청역 공영주차장(구),강남구 개포동 13-2,노외 주차장,강남구,개포동,동남권,168
4,개포동공원 공영주차장(구),강남구 개포동 183-2,노외 주차장,강남구,개포동,동남권,149


In [6]:
주차장2['lat']=주차장['lat']
주차장2['lon']=주차장['lon']
주차장2.head()

,name,address,type,gu,dong,group,park_space,lat,lon
0,구룡산제2호(구),강남구 개포동 126-2,노외 주차장,강남구,개포동,동남권,132,37.477319,127.066489
1,논현로22길(시),강남구 개포동 1266-0,노외 주차장,강남구,개포동,동남권,97,37.481443,127.048030
2,포이초교 공영주차장(구),강남구 개포동 1273-0,노외 주차장,강남구,개포동,동남권,194,37.475602,127.052321
3,대청역 공영주차장(구),강남구 개포동 13-2,노외 주차장,강남구,개포동,동남권,168,37.494945,127.079297
4,개포동공원 공영주차장(구),강남구 개포동 183-2,노외 주차장,강남구,개포동,동남권,149,37.489490,127.066202


In [7]:
cnt=[]
for i in range(len(비교['lat'])):
    for j in range(len(주차장2['lat'])):
        if 비교['lat'][i]==주차장2['lat'][j] and 비교['lon'][i]==주차장2['lon'][j]:
            cnt.append(j)
cnt

[3,
 36,
 53,
 101,
 183,
 208,
 256,
 279,
 314,
 313,
 345,
 373,
 408,
 414,
 435,
 435,
 481,
 498,
 528,
 536,
 545,
 579,
 615,
 613,
 626,
 629,
 642,
 615,
 685,
 715,
 728]

In [8]:
len(주차장2)

911

In [9]:
len(cnt)

31

In [10]:
주차장2[주차장2['lat']==37.4949449]

,name,address,type,gu,dong,group,park_space,lat,lon
3,대청역 공영주차장(구),강남구 개포동 13-2,노외 주차장,강남구,개포동,동남권,168,37.494945,127.079297


In [11]:
비교[비교['lat']==37.4949449]

,address,lon,lat,주소구분,표준신주소,표준구주소,법정동명
0,서울특별시 강남구 개포동 13-2,127.079297,37.494945,구주소,서울특별시 강남구 개포로 623-1,서울특별시 강남구 개포동 13-2,개포동


In [12]:
for i in cnt:
    주차장2=주차장2.drop([주차장2.index[i]])
len(주차장2)

880

In [13]:
911-31

880

In [14]:
비교[비교['lat']==37.4949449]

,address,lon,lat,주소구분,표준신주소,표준구주소,법정동명
0,서울특별시 강남구 개포동 13-2,127.079297,37.494945,구주소,서울특별시 강남구 개포로 623-1,서울특별시 강남구 개포동 13-2,개포동


In [15]:
주차장2[주차장2['lat']==37.4949449]

,name,address,type,gu,dong,group,park_space,lat,lon


In [16]:
주차장2[주차장2['gu']=='강서구']

,name,address,type,gu,dong,group,park_space,lat,lon
120,가양라이품주차장(시),강서구 가양동 1457-1,노외 주차장,강서구,가양동,서남권,38,32.52202,123.741774
121,영보빌딩(시),강서구 가양동 1480-10,노외 주차장,강서구,가양동,서남권,79,32.52202,123.741774
122,가양3동 황금내공원길(구),강서구 가양동 1485-0,노상 주차장,강서구,가양동,서남권,80,32.52202,123.741774
123,가양3동 노외주차장(시),강서구 가양동 1488-12,노외 주차장,강서구,가양동,서남권,56,32.52202,123.741774
124,개화역 환승주차장(시),강서구 개화동 664-0,노외 주차장,강서구,개화동,서남권,483,32.52202,123.741774
125,공항동(송정역)(시),강서구 공항동 42-8,노외 주차장,강서구,공항동,서남권,11,32.52202,123.741774
126,공항2임시(구),강서구 공항동 703-0,노외 주차장,강서구,공항동,서남권,160,32.52202,123.741774
127,방화로(시),강서구 공항동 796-0,노상 주차장,강서구,공항동,서남권,59,32.52202,123.741774
128,송화골목 전통시장 고객주차장(구),강서구 내발산동 720-4,노외 주차장,강서구,내발산동,서남권,24,32.52202,123.741774
129,등촌 2-1 공영주차장(구),강서구 등촌동 531-5,노외 주차장,강서구,등촌동,서남권,91,32.52202,123.741774


In [17]:
# 주차장2.to_csv('최종후보공영주차장.csv',index=False)

In [18]:
len(주차장2)

880